# Predicted and Actual ROI experimentation

This notebook will seek to figure out the relationship between Predicted ROI and Actual ROI. Methodology is as follows:
1. Create a baseline dataset with randomly generated `label`, `score`, and `amount` columns. 
2. Calculate TPR and TNR from baseline dataset.
3. Create a sample dataset with randomly generated `label`, `score`, and `amount` columns.
4. Using the `label` column from the sample dataset and the baseline TPR and TNR metrics, calculate `Predicted ROI`
5. Using the rest of the sample dataset, calculate `Actual ROI`
6. Compare `Predicted ROI` and `Actual ROI`

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

In [2]:
np.random.seed(777)

In [3]:
cost_matrix = {
    "TP": 1.5,
    "FP": -2,
    "TN": 2,
    "FN": -1.5,
}

In [4]:
def get_type(row):
    if row['score'] == row['label']:
        if row['score'] == 0:
            return 'TN'
        else:
            return 'TP'
    else:
        if row['score'] == 0:
            return 'FN'
        else:
            return 'FP'

In [5]:
baseline_preds = [int(np.random.rand() > 0.5) for x in range(20)]
baseline_labels = [int(np.random.rand() > 0.5) for x in range(20)]
baseline_amount = [int(np.floor(np.random.rand() * 5000)) for x in range(20)]
dicto_baseline = {'score' : baseline_preds, 'label' : baseline_labels}
df_baseline = pd.DataFrame(dicto_baseline, columns=['score', 'label'])
df_baseline['type'] = df_baseline.apply(get_type, axis=1)
df_baseline['amount'] = baseline_amount

In [6]:
baseline_metrics = classification_report(df_baseline['label'], df_baseline['score'], output_dict=True)
baseline_metrics

{'0': {'precision': 0.7,
  'recall': 0.7777777777777778,
  'f1-score': 0.7368421052631577,
  'support': 9},
 '1': {'precision': 0.8,
  'recall': 0.7272727272727273,
  'f1-score': 0.761904761904762,
  'support': 11},
 'accuracy': 0.75,
 'macro avg': {'precision': 0.75,
  'recall': 0.7525252525252526,
  'f1-score': 0.7493734335839599,
  'support': 20},
 'weighted avg': {'precision': 0.7550000000000001,
  'recall': 0.75,
  'f1-score': 0.7506265664160401,
  'support': 20}}

In [7]:
baseline_TPR = baseline_metrics['1']['recall']
baseline_TNR = baseline_metrics['0']['recall']

In [8]:
sample_preds = [int(np.random.rand() > 0.5) for x in range(20)]
sample_labels = [int(np.random.rand() > 0.5) for x in range(20)]
sample_amount = [int(np.floor(np.random.rand() * 5000)) for x in range(20)]
dicto_sample = {'score' : sample_preds, 'label' : sample_labels}
df_sample = pd.DataFrame(dicto_sample, columns=['score', 'label'])
df_sample['type'] = df_sample.apply(get_type, axis=1)
df_sample['amount'] = sample_amount

In [9]:
predicted = 0
for ix, row in df_sample.iterrows():
    if row['score'] == 0:
        predicted += row['amount'] * (baseline_TNR * cost_matrix['TN'] + (1-baseline_TNR) * cost_matrix['FN'])
    else:
        predicted += row['amount'] * (baseline_TPR * cost_matrix['TP'] + (1-baseline_TPR) * cost_matrix['FP'])
print(predicted)

40496.40404040405


At this point, assume we get the labels for the sample dataset, ie. label column is now available.

In [10]:
actual = 0
for ix, row in df_sample.iterrows():
    actual += row['amount'] * cost_matrix[row['type']]
print(actual)

16939.0


In [11]:
diff = actual - predicted
print(f'${round(diff, 2)} difference')

$-23557.4 difference


In [12]:
diff_percent = diff / predicted
print(f'{round(diff_percent * 100, 2)}% difference')

-58.17% difference


Take a look at the metrics for the sample to see if discrepancy in actual and predicted can be explained.

In [13]:
sample_metrics = classification_report(df_sample['label'], df_sample['score'], output_dict=True)
sample_metrics

{'0': {'precision': 0.6, 'recall': 0.6, 'f1-score': 0.6, 'support': 10},
 '1': {'precision': 0.6, 'recall': 0.6, 'f1-score': 0.6, 'support': 10},
 'accuracy': 0.6,
 'macro avg': {'precision': 0.6,
  'recall': 0.6,
  'f1-score': 0.6,
  'support': 20},
 'weighted avg': {'precision': 0.6,
  'recall': 0.6,
  'f1-score': 0.6,
  'support': 20}}

In [14]:
sample_TNR = sample_metrics['0']['recall']
sample_TPR = sample_metrics['1']['recall']

In [15]:
round(abs(baseline_TNR - sample_TNR), 3)

0.178

In [16]:
round(abs(baseline_TPR - sample_TPR), 3)

0.127

# Concluding remarks
Predicted ROI will get close to Actual ROI based on two factors:
1. If the predicted TPR/TNR is similar to the baseline TPR/TNR
2. to a lesser extent, how well the model performs

Essentially, to get an accurate predicted ROI, there must be little to no concept drift and, to a lesser degree, a well-performing model.